In [5]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from scipy import signal
import netCDF4 as nc 
import os 
import time as tm

start_time = tm.time()
model_counter = 1
path_ua850 = "/rds/general/project/circulates/live/data/cmip5/ua850/"

"Looping through the models"
for model in os.listdir(path_ua850):

    ds_ws = xr.open_dataset("/rds/general/project/circulates/live/data/cmip5/ua850/%s" % (model))
    ds_pressure = xr.open_dataset("/rds/general/project/circulates/live/data/cmip5/ps/%s" % ('ps' + model[5:]))
    
    if 

    '''Filling in the unfilled coordinates''' 
    #Creating equally spaced latitude array that includes real coordinates
    latss = np.empty([31*25])
    for i in range(0, 31):
        latss[(i*25):(i+1)*25] = np.linspace(ds_ws.lat[i], ds_ws.lat[i+1], 26)[0:-1]
    latss = np.append(latss, ds_ws.lat[31])

    '''Creating a new windspeed file'''
    old_lons = np.append(np.array(ds_ws.lon[:]), 360.0) #interpolating up to 360

    path_ws = "/rds/general/user/ib719/home/UROP_2021/CMIP5_historical/ws_inter_%s.nc" % (model[11:19])
    ds_ua850 = nc.Dataset(path_ws, 'w', format = 'NETCDF4')
    time = ds_ua850.createDimension('time', len(ds_ws.time))
    lat = ds_ua850.createDimension('lat', 31*25+1)
    lon = ds_ua850.createDimension('lon', 128)

    times = ds_ua850.createVariable('time', np.float32, ('time',))
    lats = ds_ua850.createVariable('lat', np.float32, ('lat',))
    lons = ds_ua850.createVariable('lon', np.float32, ('lon',))
    value = ds_ua850.createVariable('value', np.float32, ('time', 'lat', 'lon',))
    value.units = 'm/s'
    times[:] = np.array(ds_ws.time)
    lons[:] = np.array(ds_ws.lon)
    lats[:] = latss

    for i in range(0, len(ds_ws.time)):
        ds = ds_ws.isel(time=i).isel(level=0)
        values = ds.ua
        value[i,:,:] = sp.interpolate.griddata(np.array(ds_ws.lat), values, latss , method='cubic')

    ds_ua850.close()
    print("Model %s (%d/47): Calculated interpolated windspeeds" % (model[11:19], model_counter))
    
    '''Creating a new pressure file'''
    path_ps = "/rds/general/user/ib719/home/UROP_2021/CMIP5_historical/ps_inter_%s.nc" % (model[11:19])
    ps_ic = nc.Dataset(path_ps, 'w', format = 'NETCDF4')
    time = ps_ic.createDimension('time', len(ds_pressure.time))
    lat = ps_ic.createDimension('lat', 776)
    lon = ps_ic.createDimension('lon', 128)

    times = ps_ic.createVariable('time', np.float32, ('time',))
    lats = ps_ic.createVariable('lat', np.float32, ('lat',))
    lons = ps_ic.createVariable('lon', np.float32, ('lon',))
    value = ps_ic.createVariable('value', np.float32, ('time', 'lat', 'lon',))
    value.units = 'm/s'
    times[:] = ds_pressure.time[:]
    lats[:] = latss
    lons[:] = ds_pressure.lon[:]

    for i in range(0, len(ds_pressure.time)):
        ds = ds_pressure.isel(time=i)
        values = ds.ps
        value[i,:,:] = sp.interpolate.griddata(ds_ws.lat[:], values, latss[:] , method='cubic')

    ps_ic.close()
    print("Model %s (%d/47): Calculated interpolated pressure" % (model[11:19], model_counter))
    
    '''Main calculation'''
    a = xr.open_dataset("/rds/general/user/ib719/home/UROP_2021/CMIP5_historical/ws_inter_%s.nc" % (model[11:19]))
    b = xr.open_dataset("/rds/general/user/ib719/home/UROP_2021/CMIP5_historical/ps_inter_%s.nc" % (model[11:19]))
    ps_inter = b.to_array()[0, :, :, :]
    ds_ua850 = a.to_array()[0, :, 1:, :] #the 0th latitude gives nan values
    
    #Defining useful terms
    lat_ws_D = np.empty([len(ds_ws.time[0::12].dt.year), 775])  
    lat_ws_J = np.empty([len(ds_ws.time[0::12].dt.year), 775])  
    lat_ws_F = np.empty([len(ds_ws.time[0::12].dt.year), 775])

    jet_strength = np.empty([len(ds_ws.time[0::12].dt.year)])
    jet_latitude = np.empty([len(ds_ws.time[0::12].dt.year)])
    NAO_index = np.empty([len(ds_ws.time[0::12].dt.year)])
    header_param = "Year\t\tLatitude\tStrength\tNAO_index"

    #Winter mean NetCDF file
    path_winter = "/rds/general/user/ib719/home/UROP_2021/CMIP5_historical/ws_winter_%s.nc" % (model[11:19])
    avg_ws = nc.Dataset(path_winter, 'w', format = 'NETCDF4')
    time = avg_ws.createDimension('time', len(ds_ws.time))
    lat = avg_ws.createDimension('lat', 775)
    lon = avg_ws.createDimension('lon', 128)

    times = avg_ws.createVariable('time', np.float32, ('time'))
    lats = avg_ws.createVariable('lat', np.float32, ('lat'))
    lons = avg_ws.createVariable('lon', np.float32, ('lon'))
    value = avg_ws.createVariable('value', np.float32, ('time', 'lat', 'lon'))
    value.units = 'm/s'
    times[:] = np.array(ds_ws.time[0::12].dt.year)
    lats[:] = latss[1:]
    lons[:] = ds_ws.lon[:]

    #should take values from 300 (107) to 360 (128) deg i.e 0 - 60 W
    lat_ws_D[:, :] = np.mean(ds_ua850[119:1967:12, :, 107:], axis = 2) #1967 because 1968 is December
    lat_ws_J[:, :] = np.mean(ds_ua850[120:1968:12, :, 107:], axis = 2)
    lat_ws_F[:, :] = np.mean(ds_ua850[121:1968:12, :, 107:], axis = 2)

    for i in range(10, 164): # Years from 1861 to 2014 
        value[i-10, :, :] = np.mean(ds_ua850[(12*i-1):(12*i+2), :, :], axis = 0)
        azores_ps = ps_inter[(12*i-1):(12*i+2), 449, 119] # Azores DJF
        iceland_ps = ps_inter[(12*i-1):(12*i+2), 199, 121] # Iceland DJF
        NAO_index[i-10] = np.average(azores_ps - iceland_ps)/100 # Division by 100 to convert into hPa
        jet_strength[i-10] = np.average([max(lat_ws_D[i-10]), max(lat_ws_J[i-10]), max(lat_ws_F[i-10])])
        # 1 is added to the index because the first latitude in latss is not used in lat_ws_D
        jet_latitude[i-10] = np.average([latss[np.where(lat_ws_D[i-10] == max(lat_ws_D[i-10]))[0][0]+1], \
                                         latss[np.where(lat_ws_J[i-10] == max(lat_ws_J[i-10]))[0][0]+1], \
                                         latss[np.where(lat_ws_F[i-10] == max(lat_ws_F[i-10]))[0][0]+1]])

    avg_ws.close()

    params = np.column_stack([ds_ws.time[120:1957:12].dt.year, jet_latitude, jet_strength, NAO_index])
    np.savetxt('/CMIP5_historical/parameters_%s.txt' % (model[11:19]), params, fmt = '%.5f', delimiter = '\t', header = header_param, comments = '')
    print("Model %s (%d/47): Calculated winter means and parameters" % (model[11:19], model_counter))

    """ Regression Analysis """
    c = xr.open_dataset(path_winter)
    ds_avgws = c.to_array()[0, :, :, :]
    years, jet_latitude, jet_strength, NAO_index = np.loadtxt('CMIP5_historical/parameters_%s.txt' % (model[11:19]), skiprows = 1, unpack = True)
    jet_strength_de = signal.detrend(jet_strength)
    jet_latitude_de = signal.detrend(jet_latitude)
    NAO_index_de = signal.detrend(NAO_index)

    ws_transp = ds_avgws.transpose('lat', 'time', 'lon') #creating time series

    #Nao Slopes Dataset 
    path_nao_slope = "/rds/general/user/ib719/home/UROP_2021/CMIP5_historical/nao_slope_%s.nc" % (model[11:19])
    nao_slope = nc.Dataset(path_nao_slope, 'w', format = 'NETCDF4')
    lat = nao_slope.createDimension('lat', 775)
    lon = nao_slope.createDimension('lon', 128)

    lats = nao_slope.createVariable('lat', np.float32, ('lat'))
    lons = nao_slope.createVariable('lon', np.float32, ('lon'))
    value = nao_slope.createVariable('value', np.float32, ('lat', 'lon'))
    value.units = 'm/s/hPa'
    lats[:] = latss[1:]
    lons[:] = ds_ws.lon[:]

    #X should be M, Y should be M, K for polyfit 
    for i in range(0, 775):
        value[i, :] = np.array(np.polyfit(NAO_index_de, ws_transp[i, :, :], 1)[0, :]) #check the shapes

    nao_slope.close()

    #Latitude Slopes Dataset 
    path_lat_slope = "/rds/general/user/ib719/home/UROP_2021/CMIP5_historical/lat_slope_%s.nc" % (model[11:19])
    lat_slope = nc.Dataset(path_lat_slope, 'w', format = 'NETCDF4')
    lat = lat_slope.createDimension('lat', 775)
    lon = lat_slope.createDimension('lon', 128)

    lats = lat_slope.createVariable('lat', np.float32, ('lat'))
    lons = lat_slope.createVariable('lon', np.float32, ('lon'))
    value = lat_slope.createVariable('value', np.float32, ('lat', 'lon'))
    value.units = 'deg'

    lats[:] = latss[1:]
    lons[:] = ds_ws.lon[:]

    for i in range(0, 775):
        value[i, :] = np.array(np.polyfit(jet_latitude_de, ws_transp[i, :, :], 1)[0, :])

    lat_slope.close()

    #Strength Slopes Dataset
    path_str_slope = "/rds/general/user/ib719/home/UROP_2021/CMIP5_historical/str_slope_%s.nc" % (model[11:19])
    str_slope = nc.Dataset(path_str_slope, 'w', format = 'NETCDF4')
    lat = str_slope.createDimension('lat', 775)
    lon = str_slope.createDimension('lon', 128)

    lats = str_slope.createVariable('lat', np.float32, ('lat'))
    lons = str_slope.createVariable('lon', np.float32, ('lon'))
    value = str_slope.createVariable('value', np.float32, ('lat', 'lon'))
    value.units = 'm/s/deg'
    lats[:] = latss[1:]
    lons[:] = ds_ws.lon[:]

    for i in range(0, 775):
        value[i, :] = np.array(np.polyfit(jet_strength_de, ws_transp[i, :, :], 1)[0, :])

    str_slope.close()

    print("Model %s (%d/47): Slopes are saved" % (model[11:19], model_counter))

    ''' Averaging ws_winter time series '''
    path_averaged = "/rds/general/user/ib719/home/UROP_2021/CMIP5_historical/ua850_contour_%s.nc" % (model[11:19])
    cont = nc.Dataset(path_averaged, 'w', format = 'NETCDF4')
    lat = cont.createDimension('lat', 775)
    lon = cont.createDimension('lon', 128)

    lats = cont.createVariable('lat', np.float32, ('lat'))
    lons = cont.createVariable('lon', np.float32, ('lon'))
    value = cont.createVariable('value', np.float32, ('lat', 'lon'))
    value.units = 'm/s'
    lats[:] = latss[1:]
    lons[:] = ds_ws.lon[:]
    value[:, :] = np.mean(ds_avgws[:, :, :], axis = 0)

    cont.close()
    print("Model %s (%d/47): Slopes are saved, speeds are averaged" % (model[11:19], model_counter))
    
    print("Model %s (%d/47) time:  --- %s seconds ---" % (model[11:19], model_counter, tm.time() - start_time))
    model_counter += 1

print("Total time: --- %s seconds ---" % (tm.time() - start_time))

IndexError: size of data array does not conform to slice

In [50]:
ds_ws = xr.open_dataset("/rds/general/project/circulates/live/data/cmip5/ua850/ua850_Amon_ACCESS1-0_historical_r1i1p1.nc")
#ds_ws = xr.open_dataset("/rds/general/project/circulates/live/data/20CR/ua850_v2c_185101-201412.nc")
print(model, ds_ws.time[0].dt.month)

path_ua850 = "/rds/general/project/circulates/live/data/cmip5/ua850/"
for model in os.listdir(path_ua850):
    ds_ws = xr.open_dataset("/rds/general/project/circulates/live/data/cmip5/ua850/%s" % (model))
#    print(model)
    print(model, ds_ws.time[-1])

('ua850_Amon_CESM1-BGC_historical_r1i1p1.nc', <xarray.DataArray 'month' ()>
array(1)
Coordinates:
    time     datetime64[ns] 1850-01-16T12:00:00)
('ua850_Amon_CESM1-BGC_historical_r1i1p1.nc', <xarray.DataArray 'time' ()>
array(netcdftime._netcdftime.DatetimeNoLeap(2005, 12, 16, 12, 0, 0, 0, 4, 350), dtype=object)
Coordinates:
    time     object 2005-12-16 12:00:00
Attributes:
    standard_name:  time
    long_name:      time
    bounds:         time_bnds
    axis:           T)
('ua850_Amon_NorESM1-M_historical_r1i1p1.nc', <xarray.DataArray 'time' ()>
array(netcdftime._netcdftime.DatetimeNoLeap(2005, 12, 16, 12, 0, 0, 0, 4, 350), dtype=object)
Coordinates:
    time     object 2005-12-16 12:00:00
Attributes:
    standard_name:  time
    long_name:      time
    bounds:         time_bnds
    axis:           T)
('ua850_Amon_GISS-E2-R-CC_historical_r1i1p1.nc', <xarray.DataArray 'time' ()>
array(netcdftime._netcdftime.DatetimeNoLeap(2010, 12, 16, 12, 0, 0, 0, 2, 350), dtype=object)
Coordin